In [10]:
#!/usr/bin/env python

import numpy as np
import cv2

from cv_bridge import CvBridge, CvBridgeError
import rospy
from sensor_msgs.msg import Image
from std_msgs.msg import String

import yaml
import copy
import argparse
import time

from camera import Camera

def read_config(conf_file):
    data_loaded = yaml.load(conf_file)

    DIM = (data_loaded['image_width'], data_loaded['image_height'])
    rospy.loginfo("DIM:", DIM )
    camera_matrix = data_loaded['camera_matrix']
    data = camera_matrix['data']
    rows = camera_matrix['rows']
    cols = camera_matrix['cols']

    _K = []
    for i in range(rows):
        _K.append(data[i * rows:i * rows + cols:1])

    K = np.array(_K)
    rospy.loginfo("CAMERA MATRIX:\n", K)

    distortion_coefficients = data_loaded['distortion_coefficients']
    data = distortion_coefficients['data']
    rows = distortion_coefficients['rows']
    cols = distortion_coefficients['cols']

    _D = []
    for i in range(rows):
        _D.append(data[i * rows:i * rows + cols:1])

    D = np.array([[_D[0][0]], [_D[0][1]], [_D[0][2]], [_D[0][3]]])
    rospy.loginfo("DISTORION COEFFICIENTS:\n", D)

    return DIM, K, D

class CameraUndistortNode():
    def __init__(self, DIM, K, D):
        self.node = rospy.init_node('camera_undistort_node', anonymous=True)
        self.publisher_undistorted = rospy.Publisher("/undistorted_image", Image, queue_size = 1)
        self.publisher = rospy.Publisher("/recognition_image", Image, queue_size = 1)
        self.publisher_gray = rospy.Publisher("/gray_scale_image", Image, queue_size = 1)
        self.publisher_thresh = rospy.Publisher("/threshold_image", Image, queue_size = 1)
        self.publisher_contours = rospy.Publisher("/contours_image", Image, queue_size = 1)
        self.publisher_filter_contours = rospy.Publisher("/filtered_contours_image", Image, queue_size = 1)
        self.publisher_coordinates = rospy.Publisher("/pucks_coordinates", String, queue_size = 1)
        
        self.bridge = CvBridge()
        self.camera = Camera(DIM, K, D)
        self.
        
        self.subscriber = rospy.Subscriber("/usb_cam/image_raw", Image,
                                           self.__callback, queue_size = 1)
        
    def __callback(self, data):
        start_time = time.time()
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        rospy.loginfo(rospy.get_caller_id())
        
        # Process image
        cv_image = self.camera.filter_image(cv_image)
        undistorted_image = self.camera.undistort(cv_image)
        image = undistorted_image

        # Align image using field template
        if self.camera.align_image(image):
        
            # Find thresholds and contours
            image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            image_thresholds = self.camera.find_thresholds(image_gray)
            contours = self.camera.find_contours(image_gray)
            image_contours = copy.copy(image)
            image_contours = self.camera.draw_contours(image_contours, contours)

            # Filter contours
            contours_filtered = self.camera.filter_contours(contours)
            image_filter_contours = copy.copy(image)
            image_filter_contours = self.camera.draw_contours(image_filter_contours, contours_filtered)

            # Create ellipse contours around pucks 
            image_pucks = copy.copy(image)
            coordinates = self.camera.find_pucks_coordinates(contours_filtered)
            image_pucks = self.camera.draw_contours(contours_filtered, coordinates, image_pucks)

            # Publish all images to topics
            self.publisher_undistorted.publish(self.bridge.cv2_to_imgmsg(image, "bgr8"))
            self.publisher_gray.publish(self.bridge.cv2_to_imgmsg(image_gray))
            self.publisher_thresh.publish(self.bridge.cv2_to_imgmsg(image_thresholds))
            self.publisher_contours.publish(self.bridge.cv2_to_imgmsg(contours_image, "bgr8"))
            self.publisher_filter_contours.publish(self.bridge.cv2_to_imgmsg(image_filter_contours, "bgr8"))
            self.publisher.publish(self.bridge.cv2_to_imgmsg(image_pucks, "bgr8"))
            
            # Publish pucks coordinates
            self.publisher_coordinates.publish(coordinates)

            res_time = time.time()-start_time
            rospy.loginfo("RESULT TIME = ", res_time)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("-c", "--config",
                        help="path to camera config yaml file",
                        default="../configs/calibration.yaml")
    parser.add_argument("-t", "--template",
                        help="path to field's template file",
                        default="../configs/field.png")
    args = parser.parse_args()

    try:
        conf_file = open(args.config, 'r')
    except IOError as err:
        sys.exit("Couldn't find config file")
        
    DIM, K, D = read_config(conf_file)

    undistort_node = CameraUndistortNode(DIM, K, D, args.template)
    undistort_node.camera.find_vertical_projection()
    
    rospy.spin()

usage: ipykernel_launcher.py [-h] [-c CONFIG] [-t TEMPLATE]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-bcfb4b38-a727-4b05-9b06-0678e9a89406.json


SystemExit: 2

In [5]:
Cx=0.5
Cy = 1
Cx = "{0:.2f}".format(Cx)
Cy = "{0:.2f}".format(Cy)
text="Cx=""+CxCy="
print text

ValueError: Unknown format code 'f' for object of type 'str'

In [7]:
a = [1, 2, 3, 4]
b = [1, 2, 3, 4]
for i,k in zip(a,b):
    print "a"
print str(a)

a
a
a
a
[1, 2, 3, 4]


In [8]:
import skimage

def get_distances_skimage(input_img):
    lab_COLORS = skimage.color.rgb2lab(COLORS[np.newaxis, :, :])[0]
    lab_img = skimage.color.rgb2lab(input_img)
    dist_array = skimage.color.deltaE_cmc(lab_img[:, :, np.newaxis], lab_COLORS[np.newaxis, np.newaxis, :],
                                          2, 1)
    return dist_array
def determ_color(color):
    return np.argmin(get_distances_skimage(np.array([[
        color.astype(np.uint8)]]))[0, 0]), color